In [35]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from imblearn.over_sampling import SMOTE
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor

In [ ]:
pop = pd.read_csv(r'C:\Users\User\Desktop\EDurHack\Dataset\cleanData\woodchucks_with_wood_volume.csv')

pop = pop.fillna(0)

years = pop['year'].unique().tolist()

yearly_data = pop.sort_values(by='year')

lats = yearly_data['latitude'].unique().tolist()

In [ ]:
def forecast(years, lats): 
    for year in years:
        for lat in lats:
            lons = yearly_data[yearly_data['latitude'] == lat]['longitude'].unique().tolist()
            
            for lon in lons:
                location_data = yearly_data[
                    (yearly_data['latitude'] == lat) & 
                    (yearly_data['longitude'] == lon)
                ].copy()
                
                location_data = location_data.sort_values('year').reset_index(drop=True)

                location_data['time_index'] = range(len(location_data))

                location_data['lag_1'] = location_data['total_wood_chucked_lbs'].shift(1)
                location_data['lag_2'] = location_data['total_wood_chucked_lbs'].shift(2)
                location_data['lag_3'] = location_data['total_wood_chucked_lbs'].shift(3)
                
                location_data['rolling_mean_3'] = location_data['total_wood_chucked_lbs'].rolling(window=3, min_periods=1).mean()
                
                X = location_data[['year', 'time_index', 'lag_1', 'lag_2', 'lag_3', 'rolling_mean_3']]
                y = location_data['total_wood_chucked_lbs']  
            
                X_train, X_test = X, X.iloc[0:0]
                y_train, y_test = y, y.iloc[0:0]
                
                model = RandomForestRegressor(
                    n_estimators=100,
                    max_depth=10,
                    min_samples_split=2,
                    random_state=42,
                    n_jobs=-1
                )

                model.fit(X_train, y_train)  

                future_predictions = []
                last_year = location_data['year'].iloc[-1]
                last_time_index = location_data['time_index'].iloc[-1]

                last_values = location_data['total_wood_chucked_lbs'].tail(3).values
                
                for i in range(0, 16):
                    future_year = last_year + i + 1
                    future_time = last_time_index + i + 1
                    
                    future_features = {
                        'year': future_year,
                        'time_index': future_time,
                        'lag_1': last_values[-1],
                        'lag_2': last_values[-2] if len(last_values) >= 2 else last_values[-1],
                        'lag_3': last_values[-3] if len(last_values) >= 3 else last_values[-1],
                        'rolling_mean_3': np.mean(last_values[-3:]) if len(last_values) >= 3 else np.mean(last_values)
                    }
                    future_X = pd.DataFrame([future_features])
                    pred = model.predict(future_X)[0]
                    future_predictions.append(pred)
                    last_values = np.append(last_values[1:], pred) if len(last_values) >= 3 else np.append(last_values, pred)

        result_df = pd.DataFrame({
            'latitude': list(location_data['latitude']),
            'longitude': list(location_data['longitude']),
            'total_wood_chucked_lbs': list(location_data['total_wood_chucked_lbs']) + list(future_predictions),
            'type': len(location_data) + len(future_predictions)
        })

        filename = f"forecast_year{year:.4f}.csv"
        result_df.to_csv(filename, index=False)

forecast(years, lats)

TypeError: forecast() missing 1 required positional argument: 'lats'